
**Scraping Twitter**
In this notebook, we make use of Tweepy to download tweets from Twitter.

In [ ]:
!pip install tweepy

In [ ]:
import datetime
import time

import pandas as pd
import tweepy
import csv
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/MyDrive/CA683')


In [ ]:
tweepy.__version__


In [ ]:
def get_dataframe():
    return pd.DataFrame(
        columns=[
            "date",
            "username",
            "to",
            #"replies",
            "retweets",
            "favorites",
            "text",
            "mentions",
            "hashtags",
            "id",
            "permalink",
        ]
    )

In [ ]:
class TweetMiner(object):

    result_limit = 20
    data = []
    api = False

    # == OAuth Authentication ==

    twitter_keys = {
        "consumer_key": "Jn4QNKjZovPaGqAyUALV7fNdc",
        "consumer_secret": "p15WFDyHRbFgtfAgUVYh0Xh5Vy24DRxeVvCbPyAz4ILWyiFjul",
        "access_token_key": "2668453056-VY3fknrYqFHxpN5v0drgP16s6Kpfu0qiMuXcLqa",
        "access_token_secret": "fHkoh33BQ6VJ8gfWszmyFHSrFZNeOJm5RUz48QRXvSvOd"
    }

    def __init__(self, keys_dict=twitter_keys, api=api):

        self.twitter_keys = keys_dict

        auth = tweepy.OAuthHandler(
            keys_dict["consumer_key"], keys_dict["consumer_secret"]
        )
        auth.set_access_token(
            keys_dict["access_token_key"], keys_dict["access_token_secret"]
        )

        self.api = tweepy.API(
            auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True
        )
        self.twitter_keys = keys_dict

    def mine_crypto_currency_tweets(self, query):

        last_tweet_id = False
        page_num = 1

        data = get_dataframe()
        cypto_query = f"#{query}"
        print(" ===== ", query, cypto_query)
        for page in tweepy.Cursor(
            self.api.search,
            q=cypto_query,
            lang="en",
            tweet_mode="extended",
            #since = "2021-01-27",
            #until = "2021-01-28"
            count=1  # max_id=1295144957439690000,
        ).pages():
            print(" ...... new page", page_num)
            page_num += 1

            for tweet in page:
                print(tweet)
                mined = {
                    "date": tweet.created_at,
                    "username": tweet.user.screen_name,
                    "to": tweet.in_reply_to_screen_name,
                    #"replies": tweet.reply_count,
                    "retweets": tweet.retweet_count,
                    "favorites": tweet.favorite_count,
                    "text": tweet.full_text,
                    "mentions": tweet.entities["user_mentions"],
                    "hashtags": tweet.entities["hashtags"],
                    "id": tweet.id,
                    "permalink": str("https://twitter.com/" + tweet.user.screen_name + "/status/" + tweet.id_str)
                }

                data = data.append(mined, ignore_index=True)

        date_label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        print(data.head(5))
        data.to_csv("tweet.csv", index=False)
        #!cp 'tweet.csv' '/content/drive/MyDrive/CA683'


In [ ]:
miner = TweetMiner()

We will capture below data from the downloaded tweets.

In [ ]:
def main():
  miner.mine_crypto_currency_tweets(query="Bitcoin or BTC")

main()